In [3]:
!pip install implicit

     |████████████████████████████████| 18.6 MB 668 kB/s            


In [21]:
import pandas as pd
import numpy as np

import json
from scipy import sparse as sp
from tqdm.notebook import tqdm

import implicit

import tqdm

In [11]:
with open('../data/json_data_0.json') as json_file:
       json_data = json.load(json_file)

In [16]:
np.array([i['rating'] for i in json_data[0]['users']]).astype(np.float32)

array([1.        , 0.99900373, 0.99800746, 0.99701119, 0.99601491,
       0.99501863, 0.99402236, 0.99302609, 0.99202982, 0.99103357,
       0.99003733, 0.98904109, 0.98804488, 0.98704868, 0.98605249,
       0.98505633, 0.98406019, 0.98306408, 0.98206799, 0.98107193,
       0.9800759 , 0.9790799 , 0.97808394, 0.97708801, 0.97609212,
       0.97509628, 0.97410047, 0.97310471, 0.97210899, 0.97111333,
       0.97011771, 0.96912215, 0.96812664, 0.96713118, 0.96613579,
       0.96514045, 0.96414518, 0.96314997, 0.96215483, 0.96115976,
       0.96016475, 0.95916982, 0.95817496, 0.95718018, 0.95618548,
       0.95519085, 0.95419631, 0.95320185, 0.95220748, 0.9512132 ,
       0.95021901, 0.9492249 , 0.9482309 , 0.94723698, 0.94624317,
       0.94524946, 0.94425584, 0.94326233, 0.94226893, 0.94127564,
       0.94028245, 0.93928938, 0.93829642, 0.93730357, 0.93631084,
       0.93531824, 0.93432575, 0.93333339, 0.93234115, 0.93134904,
       0.93035706, 0.92936521, 0.92837349, 0.92738191, 0.92639

In [31]:
def make_coo_row(data):
    
    values = np.array([i['rating'] for i in data['users']]).astype(np.float32)
    idx = [(i['user_id'] - 100000000) for i in data['users']]
    
    return sp.coo_matrix(
        (values, ([0] * len(idx), idx)), shape=(1, 5000001),
    )

In [32]:
rows = []
for data in tqdm.tqdm(json_data):
    rows.append(make_coo_row(data))

100%|██████████| 56562/56562 [00:46<00:00, 1223.26it/s]


In [33]:
X_sparse = sp.vstack(rows).tocsr()

In [83]:
model = implicit.als.AlternatingLeastSquares(factors=8, regularization=0.0, iterations=8)

In [53]:
X_sparse

<56562x5000001 sparse matrix of type '<class 'numpy.float32'>'
	with 100613973 stored elements in Compressed Sparse Row format>

In [84]:
model.fit(X_sparse.T, show_progress=True)

/usr/local/lib/python3.6/dist-packages/implicit/utils.py:141: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 9.947456121444702 seconds
  ParameterWarning,


  0%|          | 0/8 [00:00<?, ?it/s]

In [85]:
model.save('firstmodel.npz')

In [47]:
model1 = implicit.als.AlternatingLeastSquares(factors=8, regularization=0.0, iterations=8)

In [51]:
model1 = model1.load('firstmodel.npz')

In [52]:
sp.coo_matrix(
       (1, 56562),
    ).tocsr()

In [78]:
model.recommend(0, make_coo_row1(dd).tocsr(), N=30, filter_already_liked_items=True, recalculate_user=True)

(array([30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14,
        13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  0], dtype=int32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32))

In [64]:
def make_coo_row1(data):
    if data != []:
        values = [i['rating'] for i in data]
        idx = [i['id'] for i in data]

        return sp.coo_matrix(
            (values, ([0] * len(idx), idx)), shape=(1, 56562),
        )
    else:
        return sp.coo_matrix(
            (56562)
        )